In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract

from pprint import pprint
from pdf2image import convert_from_path
from PIL import Image
from tempfile import TemporaryDirectory

import util
import lines
import group
import label
import records

In [2]:
test_files_dir = ".././test_files/"
test_files = util.list_files(test_files_dir)
test_files

['.././test_files/LS_index_1921.pdf',
 '.././test_files/LS_index_1921_ocr.pdf',
 '.././test_files/LS_index_1937.pdf',
 '.././test_files/LS_index_1986_split.pdf',
 '.././test_files/tess_test.pdf',
 '.././test_files/Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/LS_index_1976.pdf',
 '.././test_files/LS_index_1944.pdf',
 '.././test_files/LS_index_1920_split.pdf',
 '.././test_files/LS_index_1986.pdf',
 '.././test_files/LS_index_1920.pdf',
 '.././test_files/LS_index_1944_split.pdf']

In [3]:
p_start = 3
pages, dicts = util.read_pdf(test_files[0], p_start)

l_df = lines.make_lines_df(dicts)
l_df = lines.merge_close_lines(l_df)
l_df = lines.remove_useless_lines(l_df)

display(l_df)

Reading pdf from .././test_files/LS_index_1921.pdf
...
Finished reading 10 page(s)


,line_text,spans,x0,y0,x1,y1,page
0,I . CHRONOLOGICA L INDE X O F LAW S AN D ...,"[I, ., CHRONOLOGICA, L, INDE, X, O, F, LAW...",21.12,72.94,331.74,82.94,3
1,ENGLIS H EDITIO N O F TH E LEGISLATIV ...,"[ENGLIS, H, , EDITIO, N, , O, F, , TH, E, ...",21.12,85.42,331.92,95.42,3
2,"TH E INTERNATIONA L LABOU R OFFICE , 1921 .","[TH, E, INTERNATIONA, L, LABOU, R, OFFICE, ...",56.88,98.14,294.78,108.14,3
3,"Note. — T. , E. , S. , o r Tl . afte r ...","[Note., —, T., ,, E., ,, S., ,, o, r, Tl...",31.44,115.99,331.86,122.99,3
4,"text , a n extract , a summary , o r th ...","[text, ,, a, n, extract, ,, a, summary, ,,...",21.84,124.39,331.27,131.39,3
...,...,...,...,...,...,...,...
696,URUGUA Y,"[URUGUA, Y ]",63.12,505.20,96.48,510.80,12
697,Ma y 6t h Decre e issuin g regulation s ...,"[Ma, y, , 6t, h, , Decre, e, issuin, g, re...",78.00,514.71,301.94,521.71,12
698,Ac t respectin g th e genera l compulsor y...,"[Ac, t, respectin, g, th, e, genera, l, co...",132.24,523.11,301.53,530.11,12
699,rest . T I I Ur . 1,"[rest, ., T, , I, I, Ur, ., 1 ]",131.76,532.23,339.06,539.23,12


In [5]:
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(l_df)
borders = lines.make_borders_df(bins_x0, bins_x1)
display(x0_n, bins_x0, bins_x1)

3

,x0,lines,last_x0,count,page
5,"[21.12, 21.12, 21.84, 21.84, 21.84, 22.56, 22....","[0, 1, 4, 6, 7, 8, 10, 11, 16, 18]",23.52,10,3
2,"[37.68, 38.16, 38.64, 38.64, 38.64, 38.64, 38....","[12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 2...",39.12,14,3
0,"[91.68, 91.44, 91.68, 92.16, 92.64, 92.16, 92....","[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4...",92.88,41,3
3,"[70.56, 70.32, 70.08, 70.08, 70.32, 70.08, 70....","[71, 76, 78, 79, 81, 97, 105, 116, 117]",69.84,9,4
2,"[86.16, 85.92, 86.64, 85.68, 85.92, 85.92, 85....","[72, 73, 74, 75, 77, 80, 82, 98, 99, 100, 101,...",79.92,26,4
1,"[140.4, 140.16, 140.4, 140.4, 139.92, 139.92, ...","[83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 9...",140.16,50,4
2,"[24.96, 25.44, 25.44]","[157, 159, 163]",25.44,3,5
1,"[40.56, 40.56, 40.08, 40.56, 40.8, 41.04, 41.0...","[158, 160, 161, 162, 164, 165, 166, 167, 168, ...",41.52,17,5
0,"[94.56, 94.08, 94.56, 94.32, 93.84, 93.84, 94....","[177, 178, 179, 180, 181, 182, 183, 184, 185, ...",95.52,57,5
4,"[66.72, 67.92, 67.92, 67.68]","[235, 241, 244, 278]",67.68,4,6


,x1,lines,last_x1,count,page
10,"[261.42, 259.23, 261.13, 261.37, 261.66, 261.6...","[10, 29, 30, 32, 33, 34, 35, 37, 38, 39, 40, 4...",260.91,28,3
9,"[75.52, 72.81, 73.29, 73.29, 73.29, 73.29, 74....","[11, 13, 15, 17, 19, 20, 21, 22, 23, 24, 25, 2...",74.25,13,3
13,"[331.74, 331.92, 331.86, 331.27, 331.53, 331.5...","[0, 1, 3, 4, 5, 6, 7, 9]",330.46,8,3
12,"[294.78, 290.79, 290.82, 290.26, 292.16, 291.9...","[2, 31, 36, 42, 53, 55, 57]",292.98,7,3
0,"[323.42, 321.94]","[66, 69]",321.94,2,3
...,...,...,...,...,...
12,245.51,645,245.51,1,12
1,164.17,690,164.17,1,12
3,109.31,669,109.31,1,12
0,96.48,696,96.48,1,12


In [6]:
borders

,page,x0,x1,dx
0,3,22.248000,261.503214,239.255214
1,4,70.213333,120.976538,50.763205
2,5,25.280000,264.234250,238.954250
3,6,67.560000,306.344000,238.784000
4,7,41.280000,270.997083,229.717083
5,8,66.040000,304.681538,238.641538
6,9,30.960000,271.028913,240.068913
7,10,67.920000,307.082000,239.162000
8,11,33.408000,271.161892,237.753892
9,12,63.040000,301.949697,238.909697


In [7]:
li_df = label.assign_types(l_df, bins_x0, bins_x1, x0_n)
l = label.assign_labels(li_df, x0_n)
li_df.head(20)

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type
0,I . CHRONOLOGICA L INDE X O F LAW S AN D ...,"[I, ., CHRONOLOGICA, L, INDE, X, O, F, LAW...",21.12,72.94,331.74,82.94,3,0,1
1,ENGLIS H EDITIO N O F TH E LEGISLATIV ...,"[ENGLIS, H, , EDITIO, N, , O, F, , TH, E, ...",21.12,85.42,331.92,95.42,3,0,1
2,"TH E INTERNATIONA L LABOU R OFFICE , 1921 .","[TH, E, INTERNATIONA, L, LABOU, R, OFFICE, ...",56.88,98.14,294.78,108.14,3,-1,1
3,"Note. — T. , E. , S. , o r Tl . afte r ...","[Note., —, T., ,, E., ,, S., ,, o, r, Tl...",31.44,115.99,331.86,122.99,3,-1,1
4,"text , a n extract , a summary , o r th ...","[text, ,, a, n, extract, ,, a, summary, ,,...",21.84,124.39,331.27,131.39,3,0,1
5,Th e letter s an d unbrackete d number s ...,"[Th, e, letter, s, an, d, unbrackete, d, n...",30.96,133.03,331.53,140.03,3,-1,1
6,"th e Legislativ e Serie s fo r 1921 , whi...","[th, e, Legislativ, e, Serie, s, fo, r, 19...",21.84,141.43,331.50,148.43,3,0,1
7,i n questio n o r i n referrin g t o th ...,"[i, n, questio, n, o, r, i, n, referrin, g...",21.84,149.83,331.74,156.83,3,0,1
8,refe r t o page s i n th e particula r i...,"[refe, r, t, o, page, s, i, n, th, e, par...",22.56,158.23,204.79,165.23,3,0,1
9,"Th e Legislativ e Series , 1921 , havin g ...","[Th, e, Legislativ, e, Series, ,, 1921, ,, ...",31.68,166.87,330.46,173.87,3,-1,1


In [7]:
cli_df = label.correct_x0_types(li_df, bins_x0, bins_x1, x0_n)
ll_df = label.assign_labels(cli_df, x0_n)

lli_df = ll_df.copy()
lli_df["new_label"] = lli_df["label"]
lli_df = label.improve_country_classification(lli_df)
#ll_df.to_csv("labeling_1986_full.csv")

lli_df

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type,label,new_label
0,I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN,[I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN],48.0,78.66,351.96,90.66,1,-1,1,other,other
1,THE ENGLISH EDITION OF THE LEGISLATiVE,[THE ENGLISH EDITION OF THE LEGISLATiVE],75.0,91.03,351.87,104.13,1,1,1,other,other
2,SERIES OF THE INTERNATIONAL LABOUR,"[SERIES, OF, THE, INTERNATIONAL, LABOUR]",77.0,105.66,350.81,116.86,1,1,1,other,other
3,"OFFICE, 1920.","[OFFICE, 1920.]",76.0,118.41,146.62,129.81,1,1,0,other,other
4,"Nora.—T. E., S., or Ti. after the title of a l...","[Nora.—T. E., S., or Ti. after the title of a ...",63.0,147.96,350.17,157.96,1,0,1,other,other
...,...,...,...,...,...,...,...,...,...,...,...
815,Decree to include messengers under the Eight-H...,"[Decree, to include messengers under the Eight...",68.0,511.51,327.87,522.03,11,1,2,start,start
816,"October, 1920. T., Ur. 3—5 ..","[October, 1920., T.,, Ur. 3—5, ..]",85.0,520.81,225.06,531.55,11,2,0,end,end
819,. Act respecting occupational accidents. 15th ...,"[., Act, respecting occupational accidents., 1...",68.0,526.76,330.57,537.86,11,1,2,start,start
820,Ur.r .. .. .. .. .. .. .. ..,"[Ur.r, .., .., .., .., .., .., .., ..]",85.0,535.33,301.28,546.43,11,2,1,end,end


In [9]:
rec_gr = records.group_records(lli_df)
rec = records.extract_records(lli_df)
rec

,country,region,text,page
0,INTERNATIONAL,,Convention between France and Poland respectin...,1
1,INTERNATIONAL,,Labour Treaty between France and Italy. 30th S...,1
2,INTERNATIONAL,,Convention between the Minister of Foreign Aff...,1
3,INTERNATIONAL,,Convention between France and the Czecho-Slova...,1
4,INTERNATIONAL,,Convention between the Minister of Foreign Aff...,1
...,...,...,...,...
219,URUGIJAY,,Resolution concerning ocdupational accidents i...,11
220,URUGIJAY,,Decree issuing new regulations under the Act r...,11
221,URUGIJAY,,Decree respecting the supervision of hours of ...,11
222,URUGIJAY,,Decree to include messengers under the Eight-H...,11


In [10]:
rec.to_csv("index_1920.csv")

In [55]:
classification.improve_classification(ll_df)

,line_text,spans,x0,y0,x1,y1,page,x0_type,x1_type,label,new_label,date
0,1.—LIST OF LAWS AND ORDERS OF 1944 NOT PUBLISH...,[1.—LIST OF LAWS AND ORDERS OF 1944 NOT PUBLIS...,35.0,94.69,342.19,104.79,3,1,2,start,other,
1,"LEGISLATIVE SERIES FOR THAT YEAR WHICH AMEND, RE","[LEGISLATIVE SERIES FOR THAT YEAR WHICH AMEND,...",48.0,104.29,336.14,114.79,3,-1,1,other,other,
2,PEAL OR SUPERSEDE LA\VS AND ORDERS PUBLISHED PRE,[PEAL OR SUPERSEDE LA\VS AND ORDERS PUBLISHED ...,49.0,114.39,338.64,124.89,3,-1,2,other,other,
3,VIOUSLY IN THE LEGISLATIVE SERIES,[VIOUSLY IN THE LEGISLATIVE SERIES],48.0,124.46,236.91,134.86,3,-1,1,other,other,
4,ARGENTINE REPUBLIC,[ARGENTINE REPUBLIC],26.0,147.26,151.34,158.86,3,0,0,country,country,
...,...,...,...,...,...,...,...,...,...,...,...,...
4320,Apr. 28 L.D. respecting the organisation of la...,"[Apr. 28, L.D., respecting the organisation of...",62.0,528.92,369.30,539.32,82,1,2,start,start,Apr. 28
4321,agreements; labour courts]. No. 311. (Zb. pp. ...,"[agreements; labour courts]., No. 311., (Zb. p...",109.0,537.12,337.55,547.49,82,2,1,end,end,
4322,G.O. = Gacefa Oficial de los Unmdos de Venezuela.,"[G.O. =, Gacefa, Oficial de los, Unmdos de Ven...",63.0,552.82,296.01,563.62,82,1,1,other,other,
4323,Zb. = Zbornile zakona i naredaba.,"[Zb. =, Zbornile, zakona, i, naredaba.]",62.0,562.45,195.35,573.75,82,1,0,other,other,


In [77]:
pdf_path = test_files[0]
start_page = 3

pdf_pages = convert_from_path(pdf_path, 400)[start_page-1:]
pdf_df = pd.DataFrame()

for i, page_img in enumerate(pdf_pages):
    df = pytesseract.image_to_data(Image.open('test.jpg'), config="--psm 4 --dpi 400", output_type="data.frame")
    df["page_num"] = i + start_page
    pdf_df = pd.concat([pdf_df, df])

In [78]:
df = pdf_df.copy()

df = df.dropna(subset=["text"])
df = df.rename(columns={"left": "x0", "top": "y0"})
df["x1"] = df["x0"] + df["width"]
df["y1"] = df["y0"] + df["height"]
df

,level,page_num,block_num,par_num,line_num,word_num,x0,y0,width,height,conf,text,x1,y1
4,5,3,1,1,1,1,392,308,168,28,88,BELGIUM,560,336
6,5,3,1,1,2,1,478,365,79,38,96,May,557,403
7,5,3,1,1,2,2,584,366,80,34,96,24th,664,400
8,5,3,1,1,2,3,780,367,65,27,94,Act,845,394
9,5,3,1,1,2,4,869,368,34,26,96,to,903,394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,5,12,8,1,1,13,1517,3221,34,27,95,to,1551,3248
619,5,12,8,1,1,14,1577,3227,92,28,93,"1921,",1669,3255
620,5,12,8,1,1,15,1693,3229,32,27,92,p.,1725,3256
621,5,12,8,1,1,16,1752,3227,39,27,95,13,1791,3254


In [80]:
df.head(50)

,level,page_num,block_num,par_num,line_num,word_num,x0,y0,width,height,conf,text,x1,y1
4,5,3,1,1,1,1,392,308,168,28,88,BELGIUM,560,336
6,5,3,1,1,2,1,478,365,79,38,96,May,557,403
7,5,3,1,1,2,2,584,366,80,34,96,24th,664,400
8,5,3,1,1,2,3,780,367,65,27,94,Act,845,394
9,5,3,1,1,2,4,869,368,34,26,96,to,903,394
10,5,3,1,1,2,5,922,365,110,37,75,repeal,1032,402
11,5,3,1,1,2,6,1056,364,132,30,94,Section,1188,394
12,5,3,1,1,2,7,1212,373,61,27,94,310,1273,400
13,5,3,1,1,2,8,1294,367,35,28,95,of,1329,395
14,5,3,1,1,2,9,1350,365,58,30,96,the,1408,395


In [85]:
page, lines_text, x0, x1, y0, y1 = [], [], [], [], [], []

for p_no, page_frame in df.groupby("page_num"):
    for b, block in page_frame.groupby("block_num"):
        for p, par in block.groupby("par_num"):
            for no, line in par.groupby("line_num"):
                x0.append(line["x0"].min())
                x1.append(line["x1"].max())
                y0.append(line["y0"].min())
                y1.append(line["y1"].max())

                line_text = ""
                for i, word in line.iterrows():
                    line_text += word["text"] + " "
                    #print(word["text"])

                lines_text.append(line_text)
                page.append(p_no)

lines_df = pd.DataFrame({
    "line_text": lines_text,
    "x0": x0,
    "y0": y0,
    "x1": x1,
    "y1": y1,
    "page": page
})
lines_df["dx"] = lines_df["x1"] - lines_df["x0"]

lines_df

,line_text,x0,y0,x1,y1,page,dx
0,BELGIUM,392,308,560,336,3,168
1,May 24th Act to repeal Section 310 of the Pena...,478,364,2008,404,3,1530
2,May 24th Act to guarantec freedom of associati...,477,408,2007,456,3,1530
3,June 14th Act to provide for an eight-hour day...,481,455,1716,503,3,1235
4,eight hour week. T. .......-....2 0 eee e reec...,780,501,1873,548,3,1093
...,...,...,...,...,...,...,...
605,cerning relief works. S. ....... Gi io eB ave ...,774,3014,2036,3058,12,1262
606,EcuabDor,389,3065,563,3094,12,174
607,Sept. 30th Act respecting occupational acciden...,464,3122,1909,3161,12,1445
608,,399,3187,503,3192,12,104


In [86]:
lines_df.head(50)

,line_text,x0,y0,x1,y1,page,dx
0,BELGIUM,392,308,560,336,3,168
1,May 24th Act to repeal Section 310 of the Pena...,478,364,2008,404,3,1530
2,May 24th Act to guarantec freedom of associati...,477,408,2007,456,3,1530
3,June 14th Act to provide for an eight-hour day...,481,455,1716,503,3,1235
4,eight hour week. T. .......-....2 0 eee e reec...,780,501,1873,548,3,1093
5,"Aug. 7th Act of 24th December, 1903, respectin...",477,553,1713,592,3,1236
6,sation for injurics resulting from industrial,777,600,1715,639,3,938
7,"accidents, as amended by the Acts of 27th Augu...",778,648,1713,687,3,935
8,"191g, and 7th August, 921. T............... Be...",782,696,1945,736,3,1163
9,BraziL,390,746,524,774,3,134


In [47]:
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)
display(x0_n, bins_x0, bins_x1)

3

,x0,lines,last_x0,count,page
7,"[392, 390, 389, 390, 390, 390, 389, 392, 389]","[0, 9, 12, 13, 16, 18, 25, 37, 39]",389,9,1
6,"[478, 477, 481, 477, 479, 477, 476, 477, 477, ...","[1, 2, 3, 5, 10, 14, 17, 19, 20, 22, 28, 29, 3...",475,15,1
5,"[780, 777, 778, 782, 780, 780, 778, 781, 781, ...","[4, 6, 7, 8, 11, 15, 21, 23, 24, 30, 33, 36, 38]",774,13,1


,x1,lines,last_x1,count,page
19,"[1716, 1713, 1715, 1713, 1713, 1714, 1711, 171...","[3, 5, 6, 7, 10, 14, 20, 22, 23, 24, 29, 31, 3...",1708,14,1


In [48]:
lines_df.loc[bins_x1["lines"].iloc[0]]

,line_text,x0,y0,x1,y1,page,dx
3,June 14th Act to provide for an eight-hour day...,481,455,1716,503,1,1235
5,"Aug. 7th Act of 24th December, 1903, respectin...",477,553,1713,592,1,1236
6,sation for injurics resulting from industrial ...,777,600,1715,639,1,938
7,"accidents, as amended by the Acts of 27th Augu...",778,648,1713,687,1,935
10,Jan. 6th Deerce regulating alien immigration i...,479,802,1713,840,1,1234
14,Mar. 22nd Regulations for the administration o...,477,1023,1714,1060,1,1237
20,Apr. 1st Act to amend the provisions of Sectio...,477,1348,1711,1387,1,1234
22,May 21st Administrative Order of the Governmen...,478,1442,1715,1481,1,1237
23,the Czecho-Slovakian Republic under Chapter II...,781,1490,1712,1527,1,931
24,"of the Act of 11th March, 1921, No. 100, respe...",781,1538,1713,1575,1,932


In [7]:
li_df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
l = label.assign_labels(li_df, x0_n)
li_df.head(20)

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type
0,BELGIUM,392,308,560,336,1,168,0,0
1,May 24th Act to repeal Section 310 of the Pena...,478,364,2008,404,1,1530,1,1
2,May 24th Act to guarantec freedom of associati...,477,408,2007,456,1,1530,1,1
3,June 14th Act to provide for an eight-hour day...,481,455,1716,503,1,1235,1,2
4,eight hour week. T. .......-....2 0 eee e reec...,780,501,1873,548,1,1093,2,1
5,"Aug. 7th Act of 24th December, 1903, respectin...",477,553,1713,592,1,1236,1,2
6,sation for injurics resulting from industrial,777,600,1715,639,1,938,2,2
7,"accidents, as amended by the Acts of 27th Augu...",778,648,1713,687,1,935,2,2
8,"191g, and 7th August, 921. T............... Be...",782,696,1945,736,1,1163,2,1
9,BraziL,390,746,524,774,1,134,0,0


In [8]:
cli_df = label.correct_x0_types(li_df, bins_x0, bins_x1, x0_n)
ll_df = label.assign_labels(cli_df, x0_n)

lli_df = ll_df.copy()
lli_df["new_label"] = lli_df["label"]
lli_df = label.improve_country_classification(lli_df)
#ll_df.to_csv("labeling_1986_full.csv")

lli_df

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label,new_label
0,BELGIUM,392,308,560,336,1,168,0,0,country,country
1,May 24th Act to repeal Section 310 of the Pena...,478,364,2008,404,1,1530,1,1,other,other
2,May 24th Act to guarantec freedom of associati...,477,408,2007,456,1,1530,1,1,other,other
3,June 14th Act to provide for an eight-hour day...,481,455,1716,503,1,1235,1,2,start,start
4,eight hour week. T. .......-....2 0 eee e reec...,780,501,1873,548,1,1093,2,1,end,end
5,"Aug. 7th Act of 24th December, 1903, respectin...",477,553,1713,592,1,1236,1,2,start,start
6,sation for injurics resulting from industrial,777,600,1715,639,1,938,2,2,middle,middle
7,"accidents, as amended by the Acts of 27th Augu...",778,648,1713,687,1,935,2,2,middle,middle
8,"191g, and 7th August, 921. T............... Be...",782,696,1945,736,1,1163,2,1,end,end
9,BraziL,390,746,524,774,1,134,0,0,country,region


In [9]:
rec_gr = records.group_records(lli_df)
rec = records.extract_records(lli_df)
rec

,country,region,text,page
0,INTERNATIONAL,,Convention between France and Poland respectin...,1
1,INTERNATIONAL,,Labour Treaty between France and Italy. 30th S...,1
2,INTERNATIONAL,,Convention between the Minister of Foreign Aff...,1
3,INTERNATIONAL,,Convention between France and the Czecho-Slova...,1
4,INTERNATIONAL,,Convention between the Minister of Foreign Aff...,1
...,...,...,...,...
219,URUGIJAY,,Resolution concerning ocdupational accidents i...,11
220,URUGIJAY,,Decree issuing new regulations under the Act r...,11
221,URUGIJAY,,Decree respecting the supervision of hours of ...,11
222,URUGIJAY,,Decree to include messengers under the Eight-H...,11


In [8]:
lines_df.loc[9]

line_text    BraziL 
x0               390
y0               746
x1               524
y1               774
page               1
dx               134
Name: 9, dtype: object

In [11]:
pdf = pytesseract.image_to_pdf_or_hocr('test.jpg', extension='pdf')

In [12]:
with open('test.pdf', 'w+b') as f:
    f.write(pdf)

## Probleme
- Dokumente mit 2 Spalten
- Dokumente mit Länder-Überschrift über ganze Breite der Seite
- Erkennen, ab welcher Seite die eigentlichen Indexes beginnen
- Unnötigen Anfangstext auf der ersten Seite rausfiltern
    - 1920

## Fragen
